# Learning Pydantic data validation

[Using this guided tutorial series.](https://www.youtube.com/watch?v=7aBRk_JP-qY&)

## Introduction

Pydantic is important because of its **strict type** checking. It makes validation efficient and minimal.

Let's check our version to make sure we're working with at least version 2.

In [ ]:
import pydantic

print(pydantic.__version__)

: 

First, let's look at how we'd validate data in a class using the classic Python way.

In [ ]:
class Bottle:
	def __init__(self, ounces: int, drink_name="Coca-Cola") -> None:
		if not isinstance(ounces, int):
			raise TypeError(
				f"Expected ounces to be an int, got {type(ounces).__name__}"
			)

		if not isinstance(drink_name, str):
			raise TypeError(
				f"Expected name to be a str, got {type(drink_name).__name__}"
			)

		self.ounces = ounces
		self.drink_name = drink_name


try:
	sprite = Bottle(ounces="20", drink_name="Sprite")
except TypeError as e:
	print(e)

## Basics

Here's how this would be done in `pydantic`.

In [ ]:
from pydantic import BaseModel


class Bottle(BaseModel):
	ounces: int
	drink_name: str = "Coca-Cola"


sprite = Bottle(ounces="twenty", drink_name="Sprite")

`pydantic` is useful because it will attempt to convert a variable to the correct type when initializing.

In [ ]:
sprite = Bottle(ounces="20", drink_name="Sprite")
print(sprite.ounces)

We can look at the `model_fields_set` to look at the fields...

In [ ]:
print(sprite.model_fields_set)

dr_pepper = Bottle(ounces=16)
print(dr_pepper.model_fields_set)

Additionally, we have some helpers to check out our data.

In [ ]:
print("Model dump")
print(dr_pepper.model_dump())

print("Model dump json")
print(dr_pepper.model_dump_json())

print("Model json schema")
print(dr_pepper.model_json_schema())

## Nested models

Let's take a look at the `pydantic` approach to **nested models**. 

In [ ]:
from typing import List, Optional
from pydantic import BaseModel


class Drink(BaseModel):
	name: str
	price: float
	ingredients: Optional[List[str]] = None


class Bar(BaseModel):
	name: str
	location: str
	drinks: List[Drink]


bar_instance = Bar(
	name="Champagne Haven",
	location="6 Tasty St.",
	drinks=[
		{"name": "Prosecco", "price": 7.49, "ingredients": ["Champagne"]},
		{"name": "Brut", "price": 9.99},
	],
)


print(bar_instance)
print(bar_instance.model_dump())

The above works, but there's type hinting problems with `Pylance` because it's not an exact instance of a `Drink`.

In [ ]:
bar_instance = Bar(
	name="Champagne Haven",
	location="6 Tasty St.",
	drinks=[Drink(name="Prosecco", price=7.49)],
)

print(bar_instance)
print(bar_instance.model_dump())

## Additional Parsers

We can also use more advanced features of `pydantic`.

In [ ]:
!py -m pip install pydantic[email]

This includes some more advanced features to verify emails, positive integers, and more. Helpful for detailed data validation.

To veryify emails, the `EmailStr` ensures that there's a `@` sybmol and a `.com`, `.net`, etc.

In [ ]:
from typing import Annotated, List
from pydantic import BaseModel, EmailStr, PositiveInt, Field, HttpUrl, TypeAdapter


class Address(BaseModel):
	street: str
	city: str
	state: str
	zip_code: str


class Student(BaseModel):
	name: str
	major: str
	email: EmailStr


class Teacher(BaseModel):
	name: str
	email: EmailStr


class School(BaseModel):
	# The three dots after Field tell us this is a *required* property
	name: str = Field(..., pattern=r"^[a-zA-Z0-9;' ]+$")
	teacher: Teacher
	address: Address
	# This tells us that we want a list of Students minimum size of 2
	students: Annotated[List[Student], Field(min_length=2)]
	number_of_classes: PositiveInt
	online_classes: bool
	website: HttpUrl

Let's look at how we'd use this.

In [ ]:
school_instance = School(
	name="University of the People",
	teacher=Teacher(name="Bill Teach", email="bill@uopeople.com"),
	address=Address(
		street="551 Tall St.",
		city="Columbia",
		state="South Carolina",
		zip_code="29210",
	),
	students=[
		Student(
			name="Chris Wright", major="Computer Science", email="cwright@uopeople.com"
		),
		Student(
			name="Elio Ransom", major="Construction", email="eransom@uopeople.com"),
	],
	number_of_classes=12,
	online_classes=True,
	website=TypeAdapter(HttpUrl).validate_python("https://www.uopeople.com/"),
)

print(school_instance)

An important note is that each of the classes could be built using dictionaries, for example, the first student could be:

```python
students=[
	{"name": "Chris Wright", "major": "Computer Science", "email": "cwright@uopeople.com"},
	{"name": "Elio Ransom", "major": "Construction", "email": "eransom@uopeople.com"},
]
```

Left off at **9:15**.